In [10]:
import os
import numpy as np
from sklearn.model_selection import train_test_split


In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, Flatten, Dense, LSTM, TimeDistributed, Reshape

ModuleNotFoundError: No module named 'tensorflow.keras'

In [3]:
# Define the base paths for force plate (FP) and RADAR data
base_path_fp = '/Volumes/FourTBLaCie/Yoga_Study_FP_FUFD'
base_path_radar = '/Volumes/FourTBLaCie/Yoga_Study_RADAR_4Ch_FUFD'

# Assume we're dealing with the '01' subfolder as in the uploaded image example
subfolder = '01'

# Construct the full paths for the subfolders
full_path_fp = os.path.join(base_path_fp, subfolder)
full_path_radar = os.path.join(base_path_radar, subfolder)

# Get the list of files in each subfolder
files_fp = [f for f in os.listdir(full_path_fp) if f.endswith('.npy')]
files_radar = [f for f in os.listdir(full_path_radar) if f.endswith('.npy')]

# Match files by name (without the extension and the last part after underscore)
matched_files = [(fp, radar) for fp in files_fp for radar in files_radar if fp.split('_')[0] == radar.split('_')[0]]

# Load the data and create pairs
data_pairs = []
for fp, radar in matched_files:
    fp_data = np.load(os.path.join(full_path_fp, fp))
    radar_data = np.load(os.path.join(full_path_radar, radar))
    data_pairs.append((radar_data, fp_data))

# Now, you have a list of tuples where each tuple is a pair of corresponding RADAR and FP data
# Split the pairs into train and test sets
train_pairs, test_pairs = train_test_split(data_pairs, test_size=0.2, random_state=42)

# If needed, you can further split or organize these pairs into train/test folders or arrays

In [2]:
def create_model_for_cop_x(input_shape, output_sequence_length):
    input_layer = Input(shape=input_shape)  # input_shape is (4, X, 23, 13) for the RADAR RDMs
    
    # Conv3D layers to extract spatial features from the RADAR data
    x = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(input_layer)
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(x)
    x = Flatten()(x)
    
    # Dense layer for feature reduction before LSTM
    x = Dense(256, activation='relu')(x)
    # Reshaping to make it compatible for LSTM layer
    x = Reshape((output_sequence_length, -1))(x)
    
    # LSTM layer to capture temporal dynamics
    x = LSTM(128, return_sequences=True)(x)
    
    # TimeDistributed layer to predict COP_X at each time step
    output_layer = TimeDistributed(Dense(1, activation='linear'))(x)  # 1 for COP_X value
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')  # Use Mean Squared Error for regression tasks
    
    return model


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Assuming radar_data is your input RADAR RDMs with shape [num_samples, 4, X, 23, 13]
# Assuming cop_x_data is your target COP_X data with shape [num_samples, X, 1]

model = create_model_for_cop_x(input_shape=(4, X, 23, 13), output_sequence_length=X)  # X is the number of frames/time steps

# Training the model
model.fit(radar_data, cop_x_data, epochs=10, batch_size=32, validation_split=0.2)
